In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

train_X = pd.read_csv('train_X.csv', index_col=0)
train_y = pd.read_csv('train_y.csv', index_col=0)
test_X = pd.read_csv('test_X.csv', index_col=0)
sample_submission = pd.read_csv('y_sample_submission.csv')

dpps_df = pd.read_csv('dpps.csv', skiprows=1, header=None)
dpps_df.columns = dpps_df.iloc[1]
dpps_df = dpps_df.drop([0, 1]).reset_index(drop=True)

print(train_X.head())
print(test_X.head())
print(train_y.head())
print(dpps_df.head())

                                ConstructedAASeq_cln     Id
0  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...  11328
1  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...   5781
2  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...  13681
3  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...  30804
4  SKGEELFTGVVPILVELDGDVNGHTFSVSGEGEGDATYGELTLKFI...  30813
                                                       Id
ConstructedAASeq_cln                                     
SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFIC...  50579
SKGEELFTGVVPILVELDGDVSGHKFSVSGEGEGDATYGKLTLKFIC...  37987
SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFIC...  53977
SKGEELFTGVVPILVELDGDVNGHKLSVSGEGEGDATYGKLTLKFIC...  10677
SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFIC...  35653
   Brightness_Class     Id
0                 0  11328
1                 0   5781
2                 0  13681
3                 0  30804
4                 0  30813
1 AA_3 AA_1     D1     D2     D3     D4     D5     D6     D7     D8     

In [12]:
print("Columns in train_X:")
print(train_X.columns)

print("\nColumns in test_X:")
print(test_X.columns)

Columns in train_X:
Index(['ConstructedAASeq_cln', 'Id'], dtype='object')

Columns in test_X:
Index(['Id'], dtype='object')


In [4]:
amino_acid_descriptors = {}
for _, row in dpps_df.iterrows():
    amino_acid = row['AA_1']
    descriptors = row[['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10']].astype(float).tolist()
    amino_acid_descriptors[amino_acid] = descriptors



In [6]:
def sequence_to_features(seq, descriptor_dict, num_descriptors=10):
    return [descriptor_dict.get(aa, [0]*num_descriptors) for aa in seq]

In [14]:

train_X['features'] = train_X['ConstructedAASeq_cln'].apply(lambda seq: sequence_to_features(seq, amino_acid_descriptors))


In [16]:
X = np.array(train_X['features'].tolist())
y = train_y['Brightness_Class'].values

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [46]:
def scale_features(X_train, X_val):
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)
    X_train_scaled = (X_train - mean) / (std + 1e-8)  # Adding small epsilon to avoid division by zero
    X_val_scaled = (X_val - mean) / (std + 1e-8)
    return X_train_scaled, X_val_scaled

X_train_scaled, X_val_scaled = scale_features(X_train, X_val)


In [48]:

lr_model = LogisticRegression(random_state = 42)
rf_model = RandomForestClassifier(n_estimators = 100, random_state = 42)
svm_model = SVC(kernel = 'rbf', random_state = 42)



In [52]:
lr_model.fit(X_train_scaled, y_train)
lr_pred = lr_model.predict(x_val_scaled)
print("LR Accuracy:" , accuracy_score(y_val, lr_pred))


ValueError: Found array with dim 3. LogisticRegression expected <= 2.

ValueError: Found array with dim 3. RandomForestClassifier expected <= 2.